In [9]:
from utils.view_plot import *
from utils.kalman import *
from utils.quaternion import *

%matplotlib gtk

# Orientation estimation using extended kalman filter
  Based on: X. Yun and E. R. Bachmann, ‘Design, Implementation, and Experimental Results of a Quaternion-Based Kalman Filter for Human Body Motion Tracking’, IEEE Trans. Robot., vol. 22, no. 6, pp. 1216–1227, Dec. 2006, doi: 10.1109/TRO.2006.886270.
  
The state vector contains 3D angular velocity and quaternion: $$
\vec{x} = \begin{pmatrix} \omega_x & \omega_y & \omega_z & q_1 & q_2 & q_3 & q_4 \end{pmatrix}^T
$$


### Define EKF

In [2]:
kf = KalmanFilter(dim_x=7,dim_z=7)
kf.x = np.array([0,0,0,0,0,0,0])
Ts = 0.04  # 25Hz samples

kf.H = np.eye(7)

tau1 = 0.7
tau2 = 0.7
tau3 = 0.7
D = 0.0000001

q11 = (D/(2*tau1))*(1 - np.exp(-(2*Ts)/tau1))
q22 = (D/(2*tau1))*(1 - np.exp(-(2*Ts)/tau2))
q33 = (D/(2*tau1))*(1 - np.exp(-(2*Ts)/tau3))
q_quat = 0.01

kf.Q = np.array([
    [0.00456651466, -0.00102874117, 0, 0, 0, 0, 0],
    [-0.00102874117, 0.00901927163, -0.00104132302, 0, 0, 0, 0],
    [0, -0.00104132302, 0.00596346074, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]])

kf.R =  np.array([
    [0.0045610, 0, 0, 0, 0, 0, 0],
    [0, 0.00900853, 0, 0, 0, 0, 0],
    [0, 0, 0.00595636, 0, 0, 0, 0],
    [0, 0, 0, 0.001, 0, 0, 0],
    [0, 0, 0, 0, 0.001, 0, 0],
    [0, 0, 0, 0, 0, 0.001, 0],
    [0, 0, 0, 0, 0, 0, 0.001]])

kf.P *= 1000

### Load Data

In [3]:
df_stationary = pd.read_csv('../utils/straight.csv')
Ax_Bias = df_stationary['a_x'].mean()
Ay_Bias = df_stationary['a_y'].mean()
Az_Bias = df_stationary['a_z'].mean()
Gx_Bias = df_stationary['omega_x'].mean()
Gy_Bias = df_stationary['omega_y'].mean()
Gz_Bias = df_stationary['omega_z'].mean()

In [4]:
predicted_roll=[]
predicted_pitch = [] 
predicted_yaw = []
measured_roll = []
measured_pitch = []
measured_yaw = []

df = pd.read_csv('../utils/head_rotation_data/sensor_data_tilting_b.csv')
v=np.array([0.0,0,0])
for i, row in df.iterrows():
    z = [row['Gyro_X']*(np.pi/180),
         row['Gyro_Y']*(np.pi/180),
         row['Gyro_Z']*(np.pi/180),
         row['Quaternion_W'], 
         row['Quaternion_X'], 
         row['Quaternion_Y'], 
         row['Quaternion_Z']]
    
    z[3:7] = normalize_quaternion(z[3:7])
    roll, pitch, yaw = euler_from_quaternion(z[3], z[4], z[5], z[6])
    measured_roll.append(roll)
    measured_pitch.append(pitch)
    measured_yaw.append(yaw)
    
    kf.F = phi(kf.x, Ts, tau1, tau2, tau3)  
    kf.predict()
    x = kf.x
    roll, pitch, yaw = euler_from_quaternion(x[3], x[4], x[5], x[6])
    predicted_roll.append(roll)
    predicted_pitch.append(pitch)
    predicted_yaw.append(yaw)
    kf.update(z)

predicted_roll = np.array(predicted_roll)
predicted_pitch = np.array(predicted_pitch)
predicted_yaw = np.array(predicted_yaw)
measured_roll = np.array(measured_roll)
measured_pitch = np.array(measured_pitch)
measured_yaw = np.array(measured_yaw)

In [10]:
plot_roll(predicted_roll, measured_roll)
plot_roll(predicted_pitch, measured_pitch)
plot_roll(predicted_yaw, measured_yaw)

/home/wiktoria/Documents/slajdy/research and development/project/RaD_team1_HELIXON/kalman/utils/view_plot.py:432: UserWarning: Matplotlib is currently using TkAgg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
